In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("~").expanduser() / "ldap.env"
load_dotenv(dotenv_path=env_path)

In [ ]:
from ldap3 import (
    ALL,
    ALL_ATTRIBUTES,
    ALL_OPERATIONAL_ATTRIBUTES,
    AUTO_BIND_NO_TLS,
    NTLM,
    SUBTREE,
    Connection,
    Server,
)
from ldap3.core.exceptions import LDAPCursorError


class Ldaper:
    def __init__(self, server_name, base_dn, domain, user_name, password):
        self.server_name = server_name
        self.domain = domain
        self.user_name = user_name
        self.base_dn = base_dn
        self.password = password
        self.connect()

    def connect(self):
        self.server = Server(self.server_name, get_info=ALL)
        self.connection = Connection(
            self.server,
            user="{}\\{}".format(self.domain, self.user_name),
            password=self.password,
            authentication=NTLM,
            auto_bind=True,
        )

    def lookupDepartmentUsers(self, department_name, paged_size=500):
        user_filter = "(&(department={})(!(useraccountcontrol:=2)))".format(
            department_name
        )
        return self._rawLookup(user_filter, paged_size)

    def lookupUser(self, search_username, paged_size=5):
        user_filter = "(&(sAMAccountName={})(!(useraccountcontrol:=2)))".format(
            search_username
        )
        return self._rawLookup(user_filter, paged_size)

    def _rawLookup(self, query_filter, paged_size=1):
        try:
            result = self.connection.search(
                self.base_dn,
                search_filter=query_filter,
                search_scope=SUBTREE,
                attributes=ALL_ATTRIBUTES,
                paged_size=paged_size,
            )
            results = []
            for entry in self.connection.entries:
                user = entry.entry_attributes_as_dict
                results.append(user)
            return results
        except ldap3.LDAPSocketSendError:
            self.connect()
            return self.lookupUser(search_username)

In [ ]:
ld = Ldaper(
    server_name=os.environ["ldap_server_name"],
    base_dn=os.environ["ldap_base_dn"],
    user_name=os.environ["ldap_user_name"],
    password=os.environ["ldap_password"],
    domain=os.environ["ldap_domain"],
)

In [ ]:
service_accounts = []
for user in ld.lookupDepartmentUsers("ITD"):
    if "userPrincipalName" in user and "mail" in user:
        print(user["name"], user["mail"], user["userPrincipalName"])
    else:
        service_accounts.append(user)

In [ ]:
for item in service_accounts:
    print(item["name"], item["userAccountControl"])

In [ ]:
ld.lookupUser("")[0]